In [ ]:
!pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# path = "/content/drive/MyDrive/2024 K-NET 부없프/AI_Human.csv"

In [ ]:
from datasets import load_dataset

tokenized_dataset = load_dataset("cacbon-dioxit/knet-gpt-detection-project")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/614 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/197M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/197M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/197M [00:00<?, ?B/s]

valid-00000-of-00001.parquet:   0%|          | 0.00/196M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/196M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/292341 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/97447 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/97447 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'generated', 'input_ids', 'attention_mask'],
        num_rows: 292341
    })
    valid: Dataset({
        features: ['text', 'generated', 'input_ids', 'attention_mask'],
        num_rows: 97447
    })
    test: Dataset({
        features: ['text', 'generated', 'input_ids', 'attention_mask'],
        num_rows: 97447
    })
})

In [ ]:
tokenized_dataset["train"] = tokenized_dataset["train"].rename_column("generated", "labels")
tokenized_dataset["valid"] = tokenized_dataset["valid"].rename_column("generated", "labels")
tokenized_dataset["test"] = tokenized_dataset["test"].rename_column("generated", "labels")

In [ ]:
# sampling 30% (takes too much time for training)

tokenized_dataset["train"] = tokenized_dataset["train"].shuffle(seed=42).select(range(int(len(tokenized_dataset["train"]) * 0.3)))
tokenized_dataset["valid"] = tokenized_dataset["valid"].shuffle(seed=42).select(range(int(len(tokenized_dataset["valid"]) * 0.3)))
tokenized_dataset["test"] = tokenized_dataset["test"].shuffle(seed=42).select(range(int(len(tokenized_dataset["test"]) * 0.3)))

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 87702
    })
    valid: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 29234
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 29234
    })
})

In [ ]:
tokenized_dataset["train"].features # labels가 int가 아니라 float

{'text': Value(dtype='string', id=None),
 'labels': Value(dtype='float64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
from datasets import Value

# def convert_labels_to_int(example):
#     example["labels"] = int(example["labels"])
#     return example

tokenized_dataset['train'] = tokenized_dataset['train'].cast_column("labels", Value("int32"))
tokenized_dataset['valid'] = tokenized_dataset['valid'].cast_column("labels", Value("int32"))
tokenized_dataset['test'] = tokenized_dataset['test'].cast_column("labels", Value("int32"))

Casting the dataset:   0%|          | 0/87702 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/29234 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/29234 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.set_format("torch")

In [ ]:
tokenized_dataset['train'][0]['labels']

tensor(0)

In [ ]:
id2label = {0: "Human", 1: "GPT"}
label2id = {"Human": 0, "GPT": 1}

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased',
                                                            num_labels=2,
                                                            id2label=id2label,
                                                            label2id=label2id)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import evaluate
import numpy as np

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
# clf_metrics = evaluate.combine(["accuracy", "f1"])

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    clf_result = clf_metrics.compute(predictions=predictions, references=labels)
    return clf_result

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="knet-gpt-detection-project",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.017600,0.009400,0.997503,0.996663,0.996891,0.996435
2,0.003900,0.007032,0.998084,0.997443,0.996623,0.998263


TrainOutput(global_step=2742, training_loss=0.02035780432535557, metrics={'train_runtime': 9009.0489, 'train_samples_per_second': 19.47, 'train_steps_per_second': 0.304, 'total_flos': 2.3235311593857024e+16, 'train_loss': 0.02035780432535557, 'epoch': 2.0})

In [ ]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/cacbon-dioxit/knet-gpt-detection-project/commit/030a0168a36833ddeb006127f3787c9deb0e7707', commit_message='End of training', commit_description='', oid='030a0168a36833ddeb006127f3787c9deb0e7707', pr_url=None, repo_url=RepoUrl('https://huggingface.co/cacbon-dioxit/knet-gpt-detection-project', endpoint='https://huggingface.co', repo_type='model', repo_id='cacbon-dioxit/knet-gpt-detection-project'), pr_revision=None, pr_num=None)

In [ ]:
preds_output = trainer.predict(tokenized_dataset["test"])
preds_output.metrics

{'test_loss': 0.00775257870554924,
 'test_accuracy': 0.9980160087569269,
 'test_f1': 0.9973080850273832,
 'test_precision': 0.9961984237366713,
 'test_recall': 0.9984202211690363,
 'test_runtime': 435.4234,
 'test_samples_per_second': 67.139,
 'test_steps_per_second': 1.05}